- Membro 1: Giulia Monteiro Garrido (RA: 24010281)
- Membro 2: Isabella de Lima Anacleto (RA: 23906670)
- Membro 3: Maria Clara Carneiro Contiero (RA: 23916406)

---

# **CRITÉRIOS DE AVALIAÇÃO**

- Carregar base de dados supermarket (1pt)

- Implementar Regras de Associação A priori (total 5pts)

- Confiança e Lift (1pt)

- Uma métrica extra a ser pesquisada e implementada (2pts)

- Gerar uma quantidade limitada de regras de associação (qtdRegras), as regras devem ser filtradas por um limiar inferior de suporte (minSuporte), além de limiares para cada métrica (minConfianca, minLift, minExtra). Regras que não atendam os critérios não devem ser geradas. Assim que o algoritmo gerar qtdRegras fortes, deve parar e listar essas regras (2pts)

- Propor conhecimento com a base de regras gerada. (2pts)

- Explicar métricas usadas e algoritmo (2pts)



---


# **PASSO A PASSO:**

## Importando o DataFrame

1. Importando todas as bibliotecas a serem utilizadas
2. Carregando a base de dados "Supermarket"




In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

In [ ]:
!git clone https://github.com/ClaraContiero/Bases-de-Dados.git

Cloning into 'Bases-de-Dados'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 20 (delta 5), reused 4 (delta 0), pack-reused 0
Receiving objects: 100% (20/20), 622.89 KiB | 5.56 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
supermarket = pd.read_csv('/content/Bases-de-Dados/supermarket.csv')
print("\033[95mBASE DE DADOS / SUPERMARKET\n")
supermarket

BASE DE DADOS / SUPERMARKET



,department1,department2,department3,department4,department5,department6,department7,department8,department9,'grocery misc',...,department208,department209,department210,department211,department212,department213,department214,department215,department216,total
0,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,high
1,t,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
2,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
3,t,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
4,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4622,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
4623,?,?,?,t,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,high
4624,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low
4625,?,?,?,?,?,?,?,?,?,?,...,?,?,?,?,?,?,?,?,?,low


3. Calculando o tamanho dessa base de dados usando o len()
4. Podemos usar o .shape também

In [ ]:
total = len(supermarket['department1'])
print(f"\033[95mTamanho da base de dados: {total}")

Tamanho da base de dados: 4627


## Cálculos

In [ ]:
# estipulamos regras para nosso algoritmo seguir conforme for calculando as regras de associação

MIN_SUPORTE = 0.05
QTD_REGRAS = 10

In [ ]:
supermarket.shape # sendo que um é a quantidade de linhas e o outro, de colunas

(4627, 217)

5. Agora estamos criando dicionários, que associam/ligam chaves à valores. Nesse caso, criamos um dicionário VAZIO para o suporte (que é a frequência de um item em relação à base de dados).


In [ ]:
suporte = dict()
suporteUNIAO = dict()
suporteINDIVIDUAL = dict()
confiancaDICIONARIO = dict()

6. Aqui estamos guardando todas as colunas da base de dados na variável *colunas*
7. Depois, contamos quantas colunas existem em *supermarket*

In [ ]:
colunas = supermarket.columns
nColunas = len(colunas)
print(colunas)
print(f"\n\033[95mO total de colunas na base de dados Supermarket é de: {nColunas}")

Index(['department1', 'department2', 'department3', 'department4',
       'department5', 'department6', 'department7', 'department8',
       'department9', ''grocery misc'',
       ...
       'department208', 'department209', 'department210', 'department211',
       'department212', 'department213', 'department214', 'department215',
       'department216', 'total'],
      dtype='object', length=217)

O total de colunas na base de dados Supermarket é de: 217


### Cálculos de suporte

8. Aqui criamos um laço de repetição para percorrer todas as colunas da base de dados, começando em um índice 0 (que é a primeira coluna)
9. A variável *col* guarda colunas específicas com base no índice. EX: col = colunas[0] -> significa que col = primeira coluna
10. Vamos começar a calcular os suportes dos itens, ou seja, sua frequência. É basicamente contar quantas vezes ele aparece com o 't' registrado, dividir isso pelo tamanho da base e multiplicar por 100 para termos o percentual

In [ ]:
# índice das colunas
index = 0

# enquanto o índice for menor que a quantidade total de colunas
while (index < nColunas-1):
  col = colunas[index]

  # ex: supermarket.colunas[0].value_counts().get('t', 0)
  contagens = supermarket[col].value_counts().get('t', 0)

  # calculando a frequência individual
  sup = contagens/total

  if (sup != 0.0):
    suporte[col] = sup

  index += 1


In [ ]:
print("\033[95mDicionário do suporte/frequência dos dados\n\n")
suporte

Dicionário do suporte/frequência dos dados




{'department1': 0.22628052733952886,
 'department2': 0.028312081262156907,
 'department3': 0.01945104819537497,
 'department4': 0.018154311649016642,
 'department5': 0.037821482602118005,
 'department6': 0.0004322455154527772,
 'department7': 0.014480224767668035,
 'department9': 0.017722066133563863,
 "'grocery misc'": 0.03846985087529717,
 'department11': 0.001080613788631943,
 "'baby needs'": 0.13377998703263452,
 "'bread and cake'": 0.719688783228874,
 "'baking needs'": 0.6040631078452561,
 'juice-sat-cord-ms': 0.532310352280095,
 'tea': 0.19364599092284418,
 'biscuits': 0.5629997838772423,
 "'canned fish-meat'": 0.20337151502053166,
 "'canned fruit'": 0.27728549816295656,
 "'canned vegetables'": 0.3408255889345148,
 "'breakfast food'": 0.40242057488653554,
 "'cigs-tobacco pkts'": 0.15106980765074562,
 "'cigarette cartons'": 0.007996542035876377,
 'cleaners-polishers': 0.2727469202507024,
 'coffee': 0.2364382969526691,
 'sauces-gravy-pkle': 0.4756861897557813,
 'confectionary': 0.3

In [ ]:
# transformamos o dicionário em data frame
sup_individual = pd.DataFrame(data = suporte, index = [0])
sup_individual

,department1,department2,department3,department4,department5,department6,department7,department9,'grocery misc',department11,...,'750ml red imp','sparkling nz','sparkling imp','port and sherry',department193,'non host support',department210,department211,department212,department213
0,0.226281,0.028312,0.019451,0.018154,0.037821,0.000432,0.01448,0.017722,0.03847,0.001081,...,0.020964,0.02788,0.004971,0.005403,0.002161,0.018803,0.041279,0.044737,0.008213,0.004755


11. União dos suportes AUB

In [ ]:
colunas = sup_individual.columns
ncolunas = len(colunas)

for i in range(ncolunas):
  coli = colunas[i]
  for j in range(i + 1, ncolunas):
    if(i == j):
      continue
    colj = colunas[j]
    contagens = supermarket.value_counts([coli, colj])
    list(contagens.keys())
    cont = contagens.get(('t', 't'), 0)

    sup = (cont/total)

    if(sup != 0) and (sup > MIN_SUPORTE):
        suporteUNIAO[f'{coli} ∩ {colj}'] = sup

In [ ]:
print("\033[95mDicionário dos suportes encontrados com a relação de AUB\n\n")
suporteUNIAO

Dicionário dos suportes encontrados com a relação de AUB




{"department1 ∩ 'bread and cake'": 0.17160146963475253,
 "department1 ∩ 'baking needs'": 0.14977307110438728,
 'department1 ∩ juice-sat-cord-ms': 0.12751242705856927,
 'department1 ∩ biscuits': 0.14458612491895398,
 "department1 ∩ 'canned fruit'": 0.06462070456019019,
 "department1 ∩ 'canned vegetables'": 0.08450399827101794,
 "department1 ∩ 'breakfast food'": 0.09984871406959153,
 'department1 ∩ cleaners-polishers': 0.06937540523017073,
 'department1 ∩ coffee': 0.05813702182839853,
 'department1 ∩ sauces-gravy-pkle': 0.11195158850226929,
 'department1 ∩ confectionary': 0.09336503133779987,
 "department1 ∩ 'frozen foods'": 0.14350551113032203,
 'department1 ∩ jams-spreads': 0.06872703695699157,
 "department1 ∩ 'pet foods'": 0.10373892370866653,
 "department1 ∩ 'laundry needs'": 0.08882645342554571,
 "department1 ∩ 'party snack foods'": 0.12902528636265398,
 "department1 ∩ 'tissues-paper prd'": 0.12146098984223039,
 'department1 ∩ wrapping': 0.07413010590015129,
 "department1 ∩ 'soft dr

In [ ]:
df_sup_uniao = pd.DataFrame(data = suporteUNIAO, index = [0])
df_sup_uniao

,department1 ∩ 'bread and cake',department1 ∩ 'baking needs',department1 ∩ juice-sat-cord-ms,department1 ∩ biscuits,department1 ∩ 'canned fruit',department1 ∩ 'canned vegetables',department1 ∩ 'breakfast food',department1 ∩ cleaners-polishers,department1 ∩ coffee,department1 ∩ sauces-gravy-pkle,...,stationary ∩ department122,stationary ∩ 'small goods2',stationary ∩ department137,'prepared meals' ∩ department122,'prepared meals' ∩ 'small goods2','prepared meals' ∩ department137,'bake off products' ∩ department137,department122 ∩ 'small goods2',department122 ∩ department137,'small goods2' ∩ department137
0,0.171601,0.149773,0.127512,0.144586,0.064621,0.084504,0.099849,0.069375,0.058137,0.111952,...,0.07694,0.063756,0.131187,0.073482,0.066998,0.127729,0.057705,0.057921,0.118651,0.103523


### Cálculo da confiança, lift e convicção

11. Aqui vamos calcular a confiança (A -> B) e (B -> A), lift (A -> B) e a convicção (A -> B)

In [ ]:
confianca_AB = {} # criamos um dicionário para armazenar a confiança de A -> B
confianca_BA = {} # dicionário para armazenar a confiança de B -> A
lift = {} # dicionário para armazenar os valores de lift
conviccao_AB = {} # dicionário para armazenar os valores da convicção
chave = suporteUNIAO.keys() # variável pra guardar as chaves do dicionário suporteUNIAO

# loop com os itens de suporteUNIAO
for colunas in chave:

    # dividimos a string 'itemA ∩ itemB' em duas partes separadas por ' ∩ ' (:o)
    itemA, itemB = colunas.split(' ∩ ')

    # pegamos o suporte da união A e B
    supAB = df_sup_uniao[colunas][0]

    # pegamos o suporte individual de A e B
    supA = suporte[itemA]
    supB = suporte[itemB]

    # calculamos a confiança de A -> B e B -> A
    confAB = supAB / supA
    confBA = supAB / supB

    # calculamos o lift de A -> B
    liftAB = confAB / supB

    # calculamos a convicção de A -> B
    convAB = (1 - supB) / (1 - confAB)

    # adiciona nos dicionários
    confianca_AB[f'{itemA} -> {itemB}'] = confAB
    confianca_BA[f'{itemB} -> {itemA}'] = confBA
    lift[f'{itemA} -> {itemB}'] = liftAB
    conviccao_AB[f'{itemA} - > {itemB}'] = convAB


# transformamos tudo em dataframes
df_confianca_AB = pd.DataFrame(data=confianca_AB, index=['A -> B'])
df_confianca_BA = pd.DataFrame(data=confianca_BA, index=['B -> A'])

# juntando os dois dataframes da confiança
df_confianca = pd.concat([df_confianca_AB, df_confianca_BA])

df_lift = pd.DataFrame(data=lift, index=[0])
df_conviccao = pd.DataFrame(data=conviccao_AB, index=[0])

df_confianca

,department1 -> 'bread and cake',department1 -> 'baking needs',department1 -> juice-sat-cord-ms,department1 -> biscuits,department1 -> 'canned fruit',department1 -> 'canned vegetables',department1 -> 'breakfast food',department1 -> cleaners-polishers,department1 -> coffee,department1 -> sauces-gravy-pkle,...,department122 -> stationary,'small goods2' -> stationary,department137 -> stationary,department122 -> 'prepared meals','small goods2' -> 'prepared meals',department137 -> 'prepared meals',department137 -> 'bake off products','small goods2' -> department122,department137 -> department122,department137 -> 'small goods2'
A -> B,0.758357,0.661891,0.563515,0.638968,0.285578,0.373448,0.441261,0.30659,0.256925,0.494747,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
B -> A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.320144,0.306653,0.3274,0.305755,0.322245,0.31877,0.144013,0.278586,0.296117,0.25836


12. E agora, o lift

In [ ]:
# o lift já foi calculado no loop usado para a confiança!
df_lift

,department1 -> 'bread and cake',department1 -> 'baking needs',department1 -> juice-sat-cord-ms,department1 -> biscuits,department1 -> 'canned fruit',department1 -> 'canned vegetables',department1 -> 'breakfast food',department1 -> cleaners-polishers,department1 -> coffee,department1 -> sauces-gravy-pkle,...,stationary -> department122,stationary -> 'small goods2',stationary -> department137,'prepared meals' -> department122,'prepared meals' -> 'small goods2','prepared meals' -> department137,'bake off products' -> department137,department122 -> 'small goods2',department122 -> department137,'small goods2' -> department137
0,1.053729,1.095732,1.058621,1.134936,1.029905,1.095716,1.096516,1.124083,1.086645,1.04007,...,1.016682,0.973838,1.039726,1.113084,1.173115,1.160464,1.185672,1.159189,1.232132,1.242654


13. Convicção

In [ ]:
# a convicção já foi calculada no loop usado para a confiança!
df_conviccao

,department1 - > 'bread and cake',department1 - > 'baking needs',department1 - > juice-sat-cord-ms,department1 - > biscuits,department1 - > 'canned fruit',department1 - > 'canned vegetables',department1 - > 'breakfast food',department1 - > cleaners-polishers,department1 - > coffee,department1 - > sauces-gravy-pkle,...,stationary - > department122,stationary - > 'small goods2',stationary - > department137,'prepared meals' - > department122,'prepared meals' - > 'small goods2','prepared meals' - > department137,'bake off products' - > department137,department122 - > 'small goods2',department122 - > department137,'small goods2' - > department137
0,1.160023,1.171034,1.07149,1.210421,1.011607,1.052066,1.069514,1.048807,1.02757,1.037725,...,1.005306,0.99318,1.027285,1.037103,1.047603,1.120178,1.141733,1.043607,1.183714,1.193654


## Análise

### Suporte

In [ ]:
# aqui, pegamos os 10 maiores valores do suporte da união entre as colunas

max_sup_uniao = df_sup_uniao.max().nlargest(QTD_REGRAS)
print("\033[95mMaiores valores de suporte\n")
max_sup_uniao

Maiores valores de suporte



'bread and cake' ∩ milk-cream        0.505079
'bread and cake' ∩ fruit             0.502485
'bread and cake' ∩ vegetables        0.496650
fruit ∩ vegetables                   0.476983
'bread and cake' ∩ 'baking needs'    0.473525
'bread and cake' ∩ 'frozen foods'    0.460125
'bread and cake' ∩ biscuits          0.450184
milk-cream ∩ fruit                   0.440458
milk-cream ∩ vegetables              0.437649
'baking needs' ∩ vegetables          0.421223
dtype: float64

In [ ]:
# aqui, pegamos os 10 menores valores do suporte da união entre as colunas

min_sup_uniao = df_sup_uniao.min().nsmallest(QTD_REGRAS)
print("\033[95mMenores valores de suporte\n")
min_sup_uniao

Menores valores de suporte



juice-sat-cord-ms ∩ deod-disinfectant    0.050140
sauces-gravy-pkle ∩ deod-disinfectant    0.050140
'tissues-paper prd' ∩ cough-cold-pain    0.050140
'small goods' ∩ poultry                  0.050140
'dairy foods' ∩ 'bake off products'      0.050140
'frozen foods' ∩ pork                    0.050357
'frozen foods' ∩ department79            0.050357
'party snack foods' ∩ lamb               0.050357
'pkt-canned soup' ∩ cheese               0.050357
potatoes ∩ stationary                    0.050357
dtype: float64

### Confiança

In [ ]:
max_conf = df_confianca.max().nlargest(QTD_REGRAS)
print("\033[95mMaiores valores da confiança\n")
max_conf

Maiores valores da confiança



potatoes -> vegetables                    0.858311
'small goods' -> 'bread and cake'         0.835125
potatoes -> 'bread and cake'              0.832425
dishcloths-scour -> 'bread and cake'      0.812155
'canned fruit' -> 'bread and cake'        0.810600
jams-spreads -> 'bread and cake'          0.803599
'pet food' -> 'bread and cake'            0.803002
puddings-deserts -> 'bread and cake'      0.802030
'canned fish-meat' -> 'bread and cake'    0.801275
margarine -> 'bread and cake'             0.800262
dtype: float64

In [ ]:
min_conf = df_confianca.min().nsmallest(QTD_REGRAS)
print("\033[95mMenores valores da confiança\n")
min_conf

Menores valores da confiança



'bread and cake' -> 'variety misc'         0.070571
'bread and cake' -> kitchen                0.072072
'bread and cake' -> department130          0.075976
'bread and cake' -> 'health food other'    0.077778
'bread and cake' -> pork                   0.078979
fruit -> spices                            0.079338
fruit -> pork                              0.080351
milk-cream -> spices                       0.080640
milk-cream -> cough-cold-pain              0.080640
vegetables -> cough-cold-pain              0.080716
dtype: float64

<font color = #FF69B4>*** Análise**</color>

- O cálculo da confiança com dois valores nos traz informações interessantes sobre a venda de certos produtos neste supermercado. Por exemplo, 85,84% das vezes que as pessoas compraram batatas, elas também compraram outros vegetais. Isso se dá pois esses produtos provavelmente se encontram próximos na seção de legumes e vegetais.

- 83,51% das pessoas que compraram frios, como mortadela, salames, presuntos etc., também compraram pães e bolos, já que para um café da tarde ou lanche, são alimentos que se complementam bem. Por isso, seria interessante que o supermecado fizesse algumas promoções, como 'na compra de x gramas de frios, ganhe x% de desconto em pães ou bolos.

- Outro caso é em relação à compra de geleias. 80,36% dos compradores de geleia também compraram pães e bolos, provavelmente para fazer uma combinação gastronômica de sabores. O mesmo se aplica para margarina e pães/bolos.

- Agora, uma confiança baixa que na prática possui alguma relação é a de vegetais e sintomas como tosse, dor e resfriado, com 8,06%. Normalmente, a recomendação para fortalecer o sistema imunológico é se alimentar bem, o que pode prevenir doenças. Assim, o mercado poderia fazer alguma campanha voltada à saúde, para impulsionar a venda dos vegetais quando uma pessoa estiver doente.

- Quanto à confiança igual 7,6% entre pães e bolos e o departamento 130, provavelmente indica que são produtos que não se complementam muito, então quem vai ao mercado acaba passando apenas na padaria. Pode ser um corredor de produtos de limpeza, por exemplo.


### Lift

In [ ]:
max_lift = df_lift.max().nlargest(QTD_REGRAS)
print("\033[95mMaiores valores do Lift\n")
max_lift

Maiores valores do Lift



deodorants-soap -> 'dental needs'            1.827416
haircare -> 'dental needs'                   1.680874
beef -> lamb                                 1.670688
'pet foods' -> 'pet food'                    1.664604
deodorants-soap -> haircare                  1.664118
'canned fruit' -> puddings-deserts           1.615553
'canned fish-meat' -> puddings-deserts       1.559997
beef -> 'small goods2'                       1.557160
'canned vegetables' -> 'pkt-canned soup'     1.522080
'canned fish-meat' -> 'canned vegetables'    1.515355
dtype: float64

In [ ]:
min_lift = df_lift.min().nsmallest(QTD_REGRAS)
print("\033[95mMenores valores do Lift\n")
min_lift

Menores valores do Lift



'cigs-tobacco pkts' -> fruit                  0.815699
biscuits -> 'cigs-tobacco pkts'               0.843631
juice-sat-cord-ms -> 'cigs-tobacco pkts'      0.854644
'baking needs' -> 'cigs-tobacco pkts'         0.883384
'cigs-tobacco pkts' -> cheese                 0.891284
'cigs-tobacco pkts' -> 'frozen foods'         0.891690
'cigs-tobacco pkts' -> sauces-gravy-pkle      0.908258
'cigs-tobacco pkts' -> vegetables             0.916575
'cigs-tobacco pkts' -> 'party snack foods'    0.920474
'cigs-tobacco pkts' -> department137          0.921154
dtype: float64

<font color = #FF69B4>*** Análise Lift**</color>

- Analisando os padrões de compra, notamos que quem compra sabonete tem, consequentemente, o hábito de adquirir pasta de dente, considerando que nosso lift está em 1.8, indicando uma associação relativamente alta.

- Da mesma forma, as chances de quem compra produtos de cuidados com o cabelo também adquirir pasta de dente é de 1.6. Portanto, estes hábitos de consumo podem ser estrategicamente aproveitados. Seria vantajoso para o dono do supermercado posicionar estes produtos na mesma seção de 'Cuidados e Higiene Pessoal'. A mesma lógica se aplica para 'bife' e 'carne de cordeiro', assim como 'bife' e 'salame', que poderiam estar presentes na mesma seção de carnes e frios.

- Por outro lado, nos níveis mais baixos, é possível notar que existem diferenças significativas na compra de cigarros em comparação com outros produtos no supermercado. Em outras palavras, pessoas que compram cigarros têm uma probabilidade menor de levar outros produtos; assim, pode ser estratégico vendê-los diretamente no caixa do supermercado.



### Convicção

In [ ]:
max_conv = df_conviccao.max().nlargest(QTD_REGRAS)
print("\033[95mMaiores valores da convicção\n")
max_conv

Maiores valores da convicção



potatoes - > vegetables                      2.541196
dishcloths-scour - > 'tissues-paper prd'     1.551689
department79 - > vegetables                  1.526343
deod-disinfectant - > 'tissues-paper prd'    1.523024
'laundry needs' - > 'tissues-paper prd'      1.497139
'canned fruit' - > 'frozen foods'            1.467078
tea - > biscuits                             1.455584
tea - > margarine                            1.451724
jams-spreads - > margarine                   1.438849
tea - > 'tissues-paper prd'                  1.435755
dtype: float64

In [ ]:
min_conv = df_conviccao.min().nsmallest(QTD_REGRAS)
print("\033[95mMenores valores da convicção\n")
min_conv

Menores valores da convicção



'cigs-tobacco pkts' - > fruit                  0.753088
'cigs-tobacco pkts' - > 'frozen foods'         0.866496
'cigs-tobacco pkts' - > vegetables             0.870873
'cigs-tobacco pkts' - > sauces-gravy-pkle      0.923162
'cigs-tobacco pkts' - > 'party snack foods'    0.925353
'cigs-tobacco pkts' - > cheese                 0.930807
'beverages hot' - > vegetables                 0.941538
'cigs-tobacco pkts' - > department137          0.949924
'cigs-tobacco pkts' - > milk-cream             0.951514
'baby needs' - > vegetables                    0.952468
dtype: float64

<font color = #FF69B4>*** Análise Convicção**</color>


ALTOS VALORES DE CONVICÇÃO INDICAM UMA FORTE DEPENDECIA


- Alta Convicção (por exemplo, 2.54 para batatas -> vegetais): Isso indica que a presença de batatas na cesta de compras aumenta significativamente a probabilidade de que vegetais também sejam comprados. Com uma convicção de 2.54, é muito mais provável que vegetais sejam comprados se batatas já estiverem na cesta do que se batatas não estivessem. Isso sugere uma forte dependência dos vegetais na compra de batatas. Portanto, estrategicamente, colocar batatas e outros vegetais próximos uns dos outros poderia incentivar compras adicionais.


- Convicção Moderada: Um valor de convicção acima de 1, por exemplo, 1.55 para panos de prato -> 'lenços de papel', mas não extremamente alto, indica que existe uma relação positiva entre os itens, mas não tão forte quanto a implicada por uma convicção muito alta. Ainda assim, clientes que compram panos de prato têm maior probabilidade de comprar produtos de papel do que clientes que não compram panos de prato.